## **<font color="red">Memory in AI Agent with ADK</font>**
The turtorial demonstrates how to build ADK agent with Memory, including both `InMemoryMemoryService` and `vertexAIMemoryBankService`

### Prequisites
`pip install google-adk, google-cloud-aiplatform --upgrage --quiet`



In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

GOOGLE_CLOUD_LOCATION = os.getenv("GOOGLE_CLOUD_LOCATION")
GOOGLE_CLOUD_PROJECT = os.getenv("BQ_PROJECT_ID")

### *Define a simple conversational agent*
- Then ADK Runner defines what session service is used by the agent. The Session service is responsible for saving the conversation history.
- In this example, we are using the in-memory session service `InMemorySessionService`. However, you can use any session service with Memory
-If you want to use Google's pre-built memory tools, you can provide a Memory service to the Runner as well. In this example, we will start by directly invoking memory so that we can complare different services. So we're not directly setting `memory_service` in the Runner.

In [2]:
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import load_memory
from google.genai.types import Content, Part
from google.genai import types

In [3]:
APP_NAME = "memory_example_app"
MODEL = "gemini-2.5-flash"

In [4]:
agent = LlmAgent(
    model = MODEL,
    name = "Generic_QA_Agent",
    instruction="Answer the user's questions"
)

session_service = InMemorySessionService()

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)

runner
    

In [11]:
def call_agent(query, session, user_id):
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(
        user_id=user_id,
        session_id=session,
        new_message=content
    )
    
    for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)


### **Introduction to ADK Memory Service**
- ADK memory services offer an interface for creating and searching for memories. The `BaseMemoryService` defines the interface for managing this searchable, logn-term knowledge store. Its primary responsibilities are:
  - Ingesting information (`add_session_to_memory`): Taking the contents of a (usually completed) Session and adding relevant information to the long-term knowledge store.
  - Searching Information (`search_memory`): Allowing an agent (typically via a Tool) to query the knowledge store and retrieve relevant snippets or context based on a search query.
- In this notebook we will cover two topics:
  - `InMemoryMemoryService`: A in-memory service that allows you to search your conversation history to retrieve the raw dialogue using basic keyword matching.
  - `VertexAiMemoryBankService`: A remote service that remotely generates memories for "meaningful" information from user conversation.
- You can also interact directly with Memory Bank by using the Agent Engine SDK.


### **<font color="blue">InMemoryMemoryService</font>**
- `InMemoryMemoryService` persists all information in-memory. The events are not condensed, and information is not extracted from them.

In [6]:
from google.adk.memory import InMemoryMemoryService
in_memory_service = InMemoryMemoryService()
in_memory_service

### **<font color="blue">VertexAiMemoryBankService</font>**
- Memory Bank will only persist information that's meaningful for future interactions. `VertexAiMemoryBankService` provides a built-in integration between Memory Bank and ADK.
- To get started with Memory Bank, you need to first create an Agent Engine. This only takes a few seconds.
- If you don't have a GCP project, you can use <u>Vertex in Express model, which only requires an API key</u>.

In [8]:
from google.adk.memory import VertexAiMemoryBankService
import vertexai

client = vertexai.Client(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_LOCATION
)

agent_engine = client.agent_engines.create(
    config={
        "context_spec": {
            "memory_bank_config": {
                "generation_config": {
                    "model": f"projects/{GOOGLE_CLOUD_PROJECT}/locations/{GOOGLE_CLOUD_LOCATION}/publishers/google/models/gemini-2.5-flash"
                }
            }
        }
    }
)

memory_bank_service = VertexAiMemoryBankService(
    agent_engine_id=agent_engine.api_resource.name.split("/")[-1],
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_LOCATION
)


### **<font color="blue">Generating Memories</font>**
- Memories will be generated from your conversation history stored in the `Session` object. The ADK `add_session_to_memory` method is responsible for triggering memory generation for your memory service, but the exact behavior depends on which service you use:
  - `InMemoryMemoryService` persists all information, including the raw dialogue.
  - `VertexAiMemoryBankService` only persists _meaningful_ information. Not all conversations will result in generated memories. Additionally, the information will be condensed to individual, self-contained memories.
- In this example, we create two sessions:
  1. `chit_chat_session`, which is conversation that contains events that may not be meaningful for future interactions.
  2. `recommendations_session`, which is a conversation about looking for birthday presents for a family member. It includes implicit information that may be helpful for future conversations.


In [12]:
USER_ID = "My User"

chit_chat_session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID
)

call_agent(
    "what types of questions do you answer",
    chit_chat_session.id,
    user_id=USER_ID
)

# Refresh session object,
chit_chat_session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=chit_chat_session.id
)


Agent Response:  As a large language model, I can answer a wide variety of questions across many topics, including:

*   **Factual questions:** About history, science, geography, current events, famous people, etc.
*   **Explanatory questions:** Asking for definitions, concepts, how things work, or explanations of complex topics.
*   **Procedural questions:** How to do something, step-by-step instructions.
*   **Creative questions:** Generating ideas, stories, poems, or different kinds of text.
*   **Comparative questions:** Asking for similarities, differences, pros and cons.
*   **Opinion-based questions:** While I don't have personal opinions, I can provide a summary of common viewpoints or arguments on a topic.
*   **Problem-solving questions:** Helping to brainstorm solutions, analyze situations, or break down problems.
*   **Language-related questions:** Grammar, spelling, word meanings, translations, writing assistance.

Essentially, if it involves information, text, or reasonin

In [13]:
recommendations_session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID
)

call_agent(
    "I have a three year old niece. What recommendations do you have for birthday presents?",
    recommendations_session.id,
    user_id=USER_ID
)

call_agent(
    "I like the idea of a bike. Are there any age appropriate options?",
    recommendations_session.id,
    user_id=USER_ID
)

recommendations_session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=recommendations_session.id
)


Agent Response:  It's so fun shopping for a three-year-old! At this age, they are little sponges, developing rapidly, and love to explore, pretend, and create. Here are some recommendations, categorized for easy browsing:

### 1. Creative & Art Supplies

*   **Washable Art Supplies:** Think chunky crayons, large paper, washable markers, paint sticks (less messy than traditional paint), or even a set of finger paints. An easel can also be a big hit!
*   **Play-Doh or Kinetic Sand:** These are fantastic for sensory play, fine motor skills, and creative expression. Look for sets with fun tools and molds.
*   **Sticker Books or Dot Markers:** Great for creativity and fine motor development without a huge mess.

### 2. Imaginative & Pretend Play

*   **Dress-Up Clothes:** Princess dresses, superhero capes, doctor kits, chef outfits, or animal costumes can spark endless role-playing fun.
*   **Play Kitchen & Food:** If she doesn't have one, a play kitchen with a set of realistic-looking play

## **InMemoryMemoryService**
- First, let's see what memories are extracted from the conversation using `InMemoryMemoryService`
- The memories include the raw dialoague of the conversation.

In [14]:
await in_memory_service.add_session_to_memory(chit_chat_session)
await in_memory_service.add_session_to_memory(recommendations_session)

In [15]:
# Peek at the memories
in_memory_service._session_events

{'memory_example_app/My User': {'2ce6f5cd-2981-4165-acd9-6d7f05ad8a0d': [Event(model_version=None, content=Content(
     parts=[
       Part(
         text='what types of questions do you answer'
       ),
     ],
     role='user'
   ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, interaction_id=None, invocation_id='e-5bd37a73-55f1-4a72-9a4c-39df945d47d3', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=None, end_of_agent=None, agent_state=None, rewind_before_invocation_id=None), long_running_tool_ids=None, branch=None, id='2aac

In [16]:
await in_memory_service.search_memory(app_name=APP_NAME, user_id=USER_ID, query="test")

SearchMemoryResponse(memories=[])

In [17]:
await in_memory_service.search_memory(app_name=APP_NAME, user_id=USER_ID, query="What should I get my mon for mother's day")


SearchMemoryResponse(memories=[MemoryEntry(content=Content(
  parts=[
    Part(
      text='what types of questions do you answer'
    ),
  ],
  role='user'
), custom_metadata={}, id=None, author='user', timestamp='2026-02-11T16:49:41.441229'), MemoryEntry(content=Content(
  parts=[
    Part(
      text="""As a large language model, I can answer a wide variety of questions across many topics, including:

*   **Factual questions:** About history, science, geography, current events, famous people, etc.
*   **Explanatory questions:** Asking for definitions, concepts, how things work, or explanations of complex topics.
*   **Procedural questions:** How to do something, step-by-step instructions.
*   **Creative questions:** Generating ideas, stories, poems, or different kinds of text.
*   **Comparative questions:** Asking for similarities, differences, pros and cons.
*   **Opinion-based questions:** While I don't have personal opinions, I can provide a summary of common viewpoints or argume